# Homework 5
## CSCI E-82A

## Background

Robotics are becoming common place in many business situations. For example in retail sales robotics are used in warehouse management. Amazon is widely known to be a leader in this area. But, according to a recent Wall Street Journal article, UK online grocery retailer, Ocado, is rumored to have even more sophisticated order fulfilment robotics.

https://www.wsj.com/articles/how-robots-and-drones-will-change-retail-forever-1539604800   

Not surprisingly, companies like Amazon and Ocado do not disclose much information on their robotics. Nonetheless, we can be sure that sensor fusion is a significant problem. For example, sensor fusion is a significant issue with self driving cars. See for example:

https://arxiv.org/ftp/arxiv/papers/0709/0709.1099.pdf  

These types of complex robots require years, even decades, to develop and perfect. In reality, the processes used in such complex robots are decomposable into a large number of **simple single tasks**. The complete system then operates by integrating the many single tasks into a **multi-task** environment. To avoid this complexity (and finish the course in less than 10 years:) you will address a simple single task problem. 

Bob's Orchards is a premium seller of apples and pears. Bob's customers pay a substantial premium for superior fruit. To satisfy these customers, Bob's must ensure that the fruit delivered is correctly packed and perfectly ripe. 

Like many legacy industries requiring specialized human skills, Bob's is facing a talent problem. Many of the human inspectors who expertly check each piece of fruit shipped for ripeness are approaching retirement age. Management's attempts to recruit younger people to apprentice as fruit inspectors have been, well, fruitless (oh, sorry!:). Therefore, it has become imperative to find some type of automated system which can reduce the workload on the diminishing number of human inspectors. To address this problem, Bob's is deploying technology from Robots R Us.



## Scenario  

The first robotic system to be deployed at Bob's uses a multi-sensor array to determine if the fruit being shipped is at the correct ripeness. There are two sensors, a vision system that examines the fruit for spots, indicating the fruit is over ripe, and a smell sensor that determines if the fruit is not ripe enough. If either sensor indicates the fruit is bad it is sent to a human inspector. In addition customers may reject even perfect fruit for no apparent reason, whereas others seem perfectly happy with less than perfect fruit.   

The probability distributions, decisions and utilities of this system can be summarized as follows:
1. The unconditional probability distribution of the ripeness of the fruit being packed is known
2. The probability that each sensor reads correctly or not is known experimentally. A table must be constructed that gives computes the conditional probability of accepting or rejecting fruit given the actual ripeness. More on this latter.
3. The decision is then made to have a human inspector examine the fruit or not. 
4. The utility of the human inspection of the fruit. 
4. The conditional probability of a customer accepting or not good depending on if the order is good or bad. 
5. The utility of the customer accepting or rejecting an order.  

## Instructions

You have been hired as a consultant to determine the optimal decision process for the ripeness testing robot. To perform this analysis you will do the following steps:

1. Draw an influence diagram for the fruit inspection task. 
2. Use Python, with numpy and optionally Pandas, compute and compare the utility for using robot aided inspection.
3. Compare the utility of robot aided inspection to the current manual inspection process. 

## Analysis

Once you have completed your influence diagram, display it here by replacing the influence.jpg file. 

<img src="Influence.jpg" alt="Drawing" style="width:600px; height:400px"/>
<center> **Your influence diagram must go here** </center>

> **Note:** You can save your figure as a file titled Influence.jpg in the same directory as your notebook. The figure should then be visible in the notebook and in the .html you will download. 

Before you proceed, execute the code in the cell below:

In [1]:
import numpy as np
import pandas as pd

You will now perform variable elimination on your influence diagram. You will start with the conditional probability of the customer being satisfied and the order and the utility of order delivery. 

The conditional probability of the customer being satisfied is shown here:

| | good order | bad order |
|----|----|----|
| satisfied | 0.95 | 0.10 |
| not satisfied | 0.05 | 0.90 |

And the utility of the satisfied and unsatisfied customers is:

|  | Satisfied | Not Satisfied |
|----|----|----|
|Utility | 20 | -40 |

Now, given these tables, compute the marginal utility for customers with good and bad orders. 

With the first set of variables eliminated, you will deal with eliminating the second set of variables, Your goal is to find the marginal distribution of good and bad orders given the inspection, or not, of packages for suspected bad orders. Keep in mind that some bad orders will not be correctly detected. 

Most unfortunately for Bob's, Robots R U engineers have little experience with fruit processing and the sensors are known to be unreliable. The probability of a correct reading from the spot sensor is 0.9 and for the smell sensor the probability of a correct reading is 0.7. If ether sensor indicates the fruit is not ripe, we consider the fruit as bad. This leads to 4 possible result states when integrating data from these sensors:   
- The fruit is actually good, a true positive   
- Good fruit is considered bad, a false negative.   
- The fruit is actually bad, a true negative.   
- Bad fruit is considered good, a false positive.    

Since we demand agreement from both sensors that the fruit is good we must construct a table based on the following logic. This leads to the following interpretation of the sensor tuples (spots, smell); 
- Both sensors indicate good (G,G), in which case the fruit is assumed good.    
- Either or both sensors indicate the fruit is bad (G,B, B,G, BB) in which case the fruit is assumed bad. 
    
The tuple of states of the spot and smell sensors can be any of the following 4, where T = true reading, F = false reading:    
- The spot and smell sensors can both read correctly;(T,T),
- The spot sensor reads correctly, but the smell sensor reads in error; (T,F),
- The spot sensor reads in error, and the smell sensor reads correctly; (F,T),
- Both the spot and smell sensors read in error; (F,F). 

Now, the two sensors work independently, and measure quite different physical properties of the fruit, so it is a good assumption the probability of true and false readings for the two senors are statistically independent. Given this logic, the probabilities for the CPD table are computed using the probabilities of true and false readings of the spot and smell sensors, (p(spot), p(smell)), as follows:     

|Classification | G,G | G,B | B, G | B,B |
|----|----|----|----|----|
|Good| p(T) * p(T) | 0.0 | 0.0 |0.0 |
|Good as Bad | 0.0 | p(T) * p(F) | p(F) * p(T) | p(F) * p(F) |
|Bad | 0.0 | p(T)*p(T) + p(F)*p(T) + p(F) * p(T)  | p(T) *p(T) + p(T) *p(F) + p(F) * p(T) | p(T) *p(T) + p(F) *p(T) + p(T) *p(F) |
| Bad as Good | p(F) * p(F) + p(T) * p(F) + p(F) * p(T) | 0.0 | 0.0  | 0.0 |

As you might expect given the focus on quality, Bob's has developed a process to ensure fruit entering the shipping stage of its process is ripe. However, given the practical issues around picking fruit in a real world orchard, the probability that fruit at this stage of the process is ripe is 0.8. There are several ways you might represent this information in your model. Perhaps the simplest way is:

| | Good | Good as Bad | Bad | Bad as Good |
|----|----|----|----|----|
|Probability| 0.8 | 0.8 | 0.2 | 0.2 |

Given these tables, compute the marginal distribution of correct and incorrect orders, Good, Good as Bad, Bad and Bad as Good.

Question: Given the unreliability of the sensors, is more bad fruit classified as bad or more good fruit classified as bad? 

ANS: 

For the next step of this analysis you must compute the marginal utility of inspecting suspected bad fruit (Good, Good as Bad, Bad, Bad as G). The utility of such an inspection is -10. Again, there are several ways to represent the utility of inspection in your model. One possibility is:

| | Good | Good as Bad | Bad | Bad as Good |
|----|----|----|----|----|
|Utility| 0.0 | 10.0 | 10.0 | 0.0 |

Bob's human fruit inspectors are extremely dedicated and experienced. In the following you can safely assume that the probability of good fruit following the inspection is 1.0. 

There are two possibilities for the decision node, inspect the suspect fruit, or ship the order as is. To find out you will need to compute the total utility for both cases. In both cases, the steps are:
1. Compute the probabilities of delivering good or bad fruit. 
2. Compute the total utility, using the marginal utility for delivery and the marginal utility for inspection (or not) you have already computed. 

First, compute the total utility of not inspecting suspect fruit. 

Now compute the total utility if all suspect fruit is inspected.

Question: Given the reliability of the sensors and the quality of the incoming fruit, is it better to inspect the fruit?

ANS: 

As already indicated, the current process is to have humans inspect all shipments at a utility of -10. Keeping in mind that even customers who receive perfect fruit are not always satisfied, compute the utility of the current process. 

Question: Is the maximum total utility of the process with the robotic inspection better than the human only process? 

ANS: